In [1]:
import pandas as pd
import math
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ativos
tickers = [ "CPLE6.SA", "ELET3.SA",
           "CPFE3.SA", "TAEE11.SA",
           "EGIE3.SA", "CMIG4.SA",
           "NEOE3.SA", "ALUP11.SA",
           "EQTL3.SA" , "ENGI11.SA",
           "ISAE4.SA" , "COCE5.SA"
            ]
period = "10y"
interval = "1wk"

In [2]:
data_dict = {}  

for ticker in tickers:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=True)[['Close']]
    data_dict[ticker] = df  

subplot_titles=[ticker.replace(".SA", "") for ticker in tickers]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
# grafico
height=800
width=800

mma = 15
fig = make_subplots(
    rows=math.ceil(len(tickers)/2), cols=2, shared_xaxes=False, 
    subplot_titles=[ticker.replace(".SA", "") for ticker in tickers],  # Remove ".SA"
    vertical_spacing=0.1
)

for i, ticker in enumerate(tickers):
    df = data_dict[ticker][('Close', ticker)]  # Reutiliza os dados já baixados
    avg = df.rolling(window=mma, min_periods=1).mean()
    
    row = (i // 2) + 1
    col = (i % 2) + 1

    # fig.add_trace(go.Candlestick(
    #     x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close,
    #     name=ticker, showlegend=False), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df.index, y=avg, mode='lines',
        line=dict(color='blue', width=1),
        name="Média", showlegend=False), row=row, col=col)

fig.update_layout(title_text=f"<b>Construção civil</b><br><sub>mm{mma} do fechamento semanal</sub>",
                  height= height, width=width)
fig.update_xaxes(rangeslider_visible=False,
    tickformat="%Y",  # Mostrar apenas o ano
    dtick="M36",  # Intervalo de meses entre os ticks
    ticklabelmode="period"  # Coloca o rótulo no início do período)
)
fig.show()
